**Import Pandas**

In [9]:
import pandas as pd

# ----------------------------------------------------
# Load Your Data
# ----------------------------------------------------
df_2020 = pd.read_csv('2020.csv')
df_2021 = pd.read_csv('2021.csv')
print("Successfully loaded 2020.csv and 2021.csv")


# ----------------------------------------------------
# Data Cleaning and Preparation Function
# ----------------------------------------------------
def prepare_dataframe(df, year):
    # These are the correct column names from your files
    name_column = 'ProductName'
    tagline_column = 'TagLine'
    topics_column = 'Topic'
    votes_column = 'Upvotes'

    # Keep only the columns we need
    df = df[[name_column, tagline_column, topics_column, votes_column]].copy()

    # --- THIS IS THE FIX ---
    # Convert the 'Upvotes' column to a numeric type.
    # 'coerce' will turn any values that can't be converted into an empty cell (NaN).
    df[votes_column] = pd.to_numeric(df[votes_column], errors='coerce')

    # Rename columns to a standard format for the rest of the script
    df.rename(columns={
        name_column: 'name',
        tagline_column: 'tagline',
        topics_column: 'topics',
        votes_column: 'votes_count'
    }, inplace=True)

    # Extract the first topic as the primary category
    df['Primary_Category'] = df['topics'].str.split(',').str[0].str.strip()

    # Remove rows with no category or votes (this now also removes the bad data from the 'coerce' step)
    df.dropna(subset=['Primary_Category', 'votes_count'], inplace=True)

    # Add a year column
    df['year'] = year

    return df

# Run the cleaning function on both dataframes
df_2020_clean = prepare_dataframe(df_2020, 2020)
df_2021_clean = prepare_dataframe(df_2021, 2021)
print("Data cleaning complete.")


# ----------------------------------------------------
# Feature Engineering - Calculate Growth & Metrics
# ----------------------------------------------------
# Calculate metrics for 2020
metrics_2020 = df_2020_clean.groupby('Primary_Category').agg(
    Number_of_Products_2020=('name', 'count'),
    Average_Votes_2020=('votes_count', 'mean')
).reset_index()

# Calculate metrics for 2021
metrics_2021 = df_2021_clean.groupby('Primary_Category').agg(
    Number_of_Products_2021=('name', 'count'),
    Average_Votes_2021=('votes_count', 'mean')
).reset_index()

# Merge the data from both years
df_final = pd.merge(metrics_2020, metrics_2021, on='Primary_Category', how='outer')
df_final.fillna(0, inplace=True)

# Calculate Year-over-Year Growth Rate
df_final['YoY_Growth_Rate'] = ((df_final['Number_of_Products_2021'] - df_final['Number_of_Products_2020']) / (df_final['Number_of_Products_2020'] + 1)) * 100
df_final['Average_Votes_2021'] = df_final['Average_Votes_2021'].round(0)
print("Feature engineering complete.")


# ----------------------------------------------------
# Finalize and Save the Engineered Data
# ----------------------------------------------------
# Select and rename final columns for Plotly Studio
output_columns = [
    'Primary_Category', 'Number_of_Products_2021', 'Average_Votes_2021', 'YoY_Growth_Rate'
]
engineered_df = df_final[output_columns]

engineered_df = engineered_df.rename(columns={
    'Number_of_Products_2021': 'Number_of_Products',
    'Average_Votes_2021': 'Average_Votes'
})

# Save the final DataFrame to a new CSV file
engineered_df.to_csv('engineered_data.csv', index=False)

print("\n Success! Your 'engineered_data.csv' file is ready.")
print("Download it from the file browser on the left, then upload it to Plotly Studio.")

Successfully loaded 2020.csv and 2021.csv
Data cleaning complete.
Feature engineering complete.

 Success! Your 'engineered_data.csv' file is ready.
Download it from the file browser on the left, then upload it to Plotly Studio.
